In [1]:
from datetime import date, timedelta
import gc
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb

df_train = pd.read_csv(
    '../input/train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

df_test = pd.read_csv(
    "../input/test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [2]:
items = pd.read_csv(
    "../input/items.csv",
).set_index("item_nbr")

stores = pd.read_csv(
    "../input/stores.csv",
).set_index("store_nbr")

In [3]:
items.head()

,family,class,perishable
item_nbr,,,
96995,GROCERY I,1093,0
99197,GROCERY I,1067,0
103501,CLEANING,3008,0
103520,GROCERY I,1028,0
103665,BREAD/BAKERY,2712,1


In [4]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [5]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train

In [6]:
df_2017.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion
35229871,2017-01-01,25,99197,0.693147,False
35229872,2017-01-01,25,103665,2.079442,False
35229873,2017-01-01,25,105574,0.693147,False
35229874,2017-01-01,25,105857,1.609438,False
35229875,2017-01-01,25,106716,1.098612,False


In [7]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [8]:
promo_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False        True       False   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False        True       False       False   

date                2017-01-09  2017-01-10     ...      2017-08-22  \
store_nbr item_nbr                             ...                   
1         96995          False       False     ...           False   
          99197          False       False     ...           False   
          103520         False       False     ...           False   
          103665         False       False     ...           False   
          105574         False       False     ...           False   

date                2017-08-23  2017-08-24  2017-08-25  2017-08-26  \
store_nbr item_nbr                                                   
1         96995          False       False       False       False   
          99197          False       False       False       False   
          103520         False       False       False       False   
          103665         False       False       False       False   
          105574         False       False       False       False   

date                2017-08-27  2017-08-28  2017-08-29  2017-08-30  2017-08-31  
store_nbr item_nbr                                                              
1         96995          False       False       False       False       False  
          99197          False       False       False       False       False  
          103520         False       False       False       False       False  
          103665         False       False       False       False       False  
          105574         False       False       False       False       False  

[5 rows x 243 columns]

In [9]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [10]:
df_2017.head()

date                2017-01-01  2017-01-02  2017-01-03  2017-01-04  \
store_nbr item_nbr                                                   
1         96995            0.0    0.000000    0.000000    0.000000   
          99197            0.0    0.000000    1.386294    0.693147   
          103520           0.0    0.693147    1.098612    0.000000   
          103665           0.0    0.000000    0.000000    1.386294   
          105574           0.0    0.000000    1.791759    2.564949   

date                2017-01-05  2017-01-06  2017-01-07  2017-01-08  \
store_nbr item_nbr                                                   
1         96995       0.000000    0.000000    0.000000    0.000000   
          99197       0.693147    0.693147    1.098612    0.000000   
          103520      1.098612    1.386294    0.693147    0.000000   
          103665      1.098612    1.098612    0.693147    1.098612   
          105574      2.302585    1.945910    1.609438    1.098612   

date                2017-01-09  2017-01-10     ...      2017-08-06  \
store_nbr item_nbr                             ...                   
1         96995       0.000000    0.000000     ...        1.098612   
          99197       0.000000    0.693147     ...        0.000000   
          103520      0.693147    0.693147     ...        0.000000   
          103665      0.000000    2.079442     ...        0.693147   
          105574      1.386294    2.302585     ...        0.000000   

date                2017-08-07  2017-08-08  2017-08-09  2017-08-10  \
store_nbr item_nbr                                                   
1         96995       1.098612    0.000000    0.000000    0.693147   
          99197       1.098612    0.000000    1.098612    0.000000   
          103520      0.000000    1.386294    0.000000    1.386294   
          103665      1.098612    0.000000    2.079442    2.302585   
          105574      1.791759    2.079442    1.945910    2.397895   

date                2017-08-11  2017-08-12  2017-08-13  2017-08-14  2017-08-15  
store_nbr item_nbr                                                              
1         96995       0.000000    0.000000    0.000000    0.000000    0.000000  
          99197       0.000000    0.000000    0.000000    0.000000    0.000000  
          103520      0.693147    0.693147    0.693147    0.000000    0.000000  
          103665      1.098612    0.000000    0.000000    0.693147    0.693147  
          105574      1.791759    1.791759    0.000000    1.386294    1.609438  

[5 rows x 227 columns]

In [11]:
items['class'] = items['class'].astype('category')

In [12]:
items = pd.get_dummies(items)

In [13]:
items.head()

,perishable,family_AUTOMOTIVE,family_BABY CARE,family_BEAUTY,family_BEVERAGES,family_BOOKS,family_BREAD/BAKERY,family_CELEBRATION,family_CLEANING,family_DAIRY,...,class_6920,class_6922,class_6924,class_6936,class_6954,class_6960,class_7002,class_7016,class_7034,class_7780
item_nbr,,,,,,,,,,,,,,,,,,,,,
96995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99197,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103501,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
103520,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
103665,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
items = items.reindex(df_2017.index.get_level_values(1))

In [15]:
items.shape

(167515, 371)

In [16]:
stores.head()

,city,state,type,cluster
store_nbr,,,,
1,Quito,Pichincha,D,13
2,Quito,Pichincha,D,13
3,Quito,Pichincha,D,8
4,Quito,Pichincha,D,9
5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [17]:
stores['cluster'] = stores.cluster.astype('category')

In [18]:
stores = pd.get_dummies(stores)

In [19]:
stores = stores.reindex(df_2017.index.get_level_values(0))

In [20]:
stores.head()

,city_Ambato,city_Babahoyo,city_Cayambe,city_Cuenca,city_Daule,city_El Carmen,city_Esmeraldas,city_Guaranda,city_Guayaquil,city_Ibarra,...,cluster_8,cluster_9,cluster_10,cluster_11,cluster_12,cluster_13,cluster_14,cluster_15,cluster_16,cluster_17
store_nbr,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [21]:
stores.shape

(167515, 60)

In [22]:
#sum of sales for each item across all stores for each day
df_2017_item = df_2017.groupby('item_nbr')[df_2017.columns].sum()

In [23]:
df_2017_item.shape

(4018, 227)

In [24]:
#sum of promotion for each item across all stores for each day
promo_2017_item = promo_2017.groupby('item_nbr')[promo_2017.columns].sum()

In [25]:
def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [31]:
def prepare_dataset(df, promo_df, t2017, is_train=True, name_prefix=None):
    X = {
        "promo_7_2017": get_timespan(promo_df, t2017, 7, 7).sum(axis=1).values,
        "promo_14_2017": get_timespan(promo_df, t2017, 14, 14).sum(axis=1).values,
        "promo_30_2017": get_timespan(promo_df, t2017, 30, 30).sum(axis=1).values,
        "promo_3_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 3).sum(axis=1).values,
        "promo_7_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 7).sum(axis=1).values,
        "promo_14_2017_aft": get_timespan(promo_df, t2017 + timedelta(days=15), 14, 14).sum(axis=1).values,
    }

    for i in [3, 7, 14, 30]:
        tmp1 = get_timespan(df, t2017, i, i)
        tmp2 = (get_timespan(promo_df, t2017, i, i) > 0) * 1

        X['has_promo_mean_%s' % i] = (tmp1 * tmp2.replace(0, np.nan)).mean(axis=1).values
        X['no_promo_mean_%s' % i] = (tmp1 * (1 - tmp2).replace(0, np.nan)).mean(axis=1).values
                
        
    for i in [3, 7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['diff_%s_mean' % i] = tmp.diff(axis=1).mean(axis=1).values
        X['mean_%s' % i] = tmp.mean(axis=1).values
        X['median_%s' % i] = tmp.median(axis=1).values
        X['min_%s' % i] = tmp.min(axis=1).values
        X['max_%s' % i] = tmp.max(axis=1).values
        X['std_%s' % i] = tmp.std(axis=1).values

    for i in [7, 14, 30]:
        tmp = get_timespan(df, t2017, i, i)
        X['has_sales_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_sales_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_sales_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

        tmp = get_timespan(promo_df, t2017, i, i)
        X['has_promo_days_in_last_%s' % i] = (tmp > 0).sum(axis=1).values
        X['last_has_promo_day_in_last_%s' % i] = i - ((tmp > 0) * np.arange(i)).max(axis=1).values
        X['first_has_promo_day_in_last_%s' % i] = ((tmp > 0) * np.arange(i, 0, -1)).max(axis=1).values

    tmp = get_timespan(promo_df, t2017 + timedelta(days=15), 14, 14)
    X['has_promo_days_in_after_14_days'] = (tmp > 0).sum(axis=1).values
    X['last_has_promo_day_in_after_14_days'] = i - ((tmp > 0) * np.arange(14)).max(axis=1).values
    X['first_has_promo_day_in_after_14_days'] = ((tmp > 0) * np.arange(14, 0, -1)).max(axis=1).values

    for i in range(1, 15):
        X['day_%s_2017' % i] = get_timespan(df, t2017, i, 1).values.ravel()

    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df, t2017, 140-i, 20, freq='7D').mean(axis=1).values        
        
    for i in range(-14, 15):
        X["promo_{}".format(i)] = promo_df[t2017 + timedelta(days=i)].values.astype(np.uint8)

    X = pd.DataFrame(X)

    if is_train:
        y = df[
            pd.date_range(t2017, periods=15)
        ].values
        return X, y
    if name_prefix is not None:
        X.columns = ['%s_%s' % (name_prefix, c) for c in X.columns]
    return X

In [27]:
from tqdm import tqdm

In [32]:
print("Preparing dataset...")
t2017 = date(2017, 6, 13)
num_days = 4
X_l, y_l = [], []
for i in tqdm(range(num_days)):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(df_2017, promo_2017, t2017 + delta)

    X_tmp = pd.concat([X_tmp, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)
    X_l.append(X_tmp)
    y_l.append(y_tmp)

X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)

del X_l, y_l
gc.collect()



  0%|          | 0/4 [00:00<?, ?it/s]

Preparing dataset...




 25%|██▌       | 1/4 [00:16<00:49, 16.59s/it]

 50%|█████     | 2/4 [00:33<00:33, 16.59s/it]

 75%|███████▌  | 3/4 [00:49<00:16, 16.55s/it]

100%|██████████| 4/4 [01:06<00:00, 16.64s/it]



89

### Test with project date

In [34]:
X_val, y_val = prepare_dataset(df_2017, promo_2017, date(2017, 7, 11))

X_val = pd.concat([X_val, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

X_test,y_test = prepare_dataset(df_2017, promo_2017, date(2017, 8, 1))

X_test = pd.concat([X_test, items.reset_index(drop=True), stores.reset_index(drop=True)], axis=1)

In [33]:
X_train.shape, y_train.shape

((670060, 547), (670060, 15))

In [35]:
X_val.shape, y_val.shape

((167515, 547), (167515, 15))

In [36]:
X_test.shape,y_test.shape

((167515, 547), (167515, 15))

In [37]:
import datetime

In [38]:
print("Training and predicting models...")

params = {
    'num_leaves': 80,
    'objective': 'regression',
    'min_data_in_leaf': 200,
    'learning_rate': 0.02,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'metric': 'l2',
    'num_threads': 16
}

MAX_ROUNDS = 5000
val_pred = []
test_pred = []
cate_vars = []
for i in range(15):
    start = datetime.datetime.now()
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    dtrain = lgb.Dataset(
        X_train, label=y_train[:, i],
        categorical_feature=cate_vars,
        weight=pd.concat([items["perishable"]] * num_days) * 0.25 + 1
    )
    dval = lgb.Dataset(
        X_val, label=y_val[:, i], reference=dtrain,
        weight=items["perishable"] * 0.25 + 1,
        categorical_feature=cate_vars)
    bst = lgb.train(
        params, dtrain, num_boost_round=MAX_ROUNDS,
        valid_sets=[dtrain, dval], early_stopping_rounds=125, verbose_eval=50
    )
    print("\n".join(("%s: %.2f" % x) for x in sorted(
        zip(X_train.columns, bst.feature_importance("gain")),
        key=lambda x: x[1], reverse=True
    )))
    val_pred.append(bst.predict(
        X_val, num_iteration=bst.best_iteration or MAX_ROUNDS))
    test_pred.append(bst.predict(
        X_test, num_iteration=bst.best_iteration or MAX_ROUNDS))
    end = datetime.datetime.now()
    print(f'Run Time: {end-start}')


Training and predicting models...
Step 1


/opt/conda/envs/fastai/lib/python3.6/site-packages/lightgbm/basic.py:1243: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.423573	valid_1's l2: 0.410645
[100]	training's l2: 0.325874	valid_1's l2: 0.319626
[150]	training's l2: 0.308468	valid_1's l2: 0.305588
[200]	training's l2: 0.30351	valid_1's l2: 0.302621
[250]	training's l2: 0.300738	valid_1's l2: 0.30143
[300]	training's l2: 0.298525	valid_1's l2: 0.300532
[350]	training's l2: 0.296801	valid_1's l2: 0.300017
[400]	training's l2: 0.29527	valid_1's l2: 0.299678
[450]	training's l2: 0.294017	valid_1's l2: 0.299518
[500]	training's l2: 0.292806	valid_1's l2: 0.299311
[550]	training's l2: 0.291642	valid_1's l2: 0.299204
[600]	training's l2: 0.290534	valid_1's l2: 0.299086
[650]	training's l2: 0.289456	valid_1's l2: 0.298989
[700]	training's l2: 0.288416	valid_1's l2: 0.298924
[750]	training's l2: 0.28741	valid_1's l2: 0.298861
[800]	training's l2: 0.286465	valid_1's l2: 0.298831
[850]	training's l2: 0.285528	valid_1's l2: 0.298782
[900]	training's l2: 0.28463	valid_1's l2:

Run Time: 0:02:57.422397
Step 2
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.423626	valid_1's l2: 0.40972
[100]	training's l2: 0.323462	valid_1's l2: 0.316471
[150]	training's l2: 0.305771	valid_1's l2: 0.302134
[200]	training's l2: 0.300358	valid_1's l2: 0.298721
[250]	training's l2: 0.297451	valid_1's l2: 0.297333
[300]	training's l2: 0.295052	valid_1's l2: 0.296332
[350]	training's l2: 0.2933	valid_1's l2: 0.295752
[400]	training's l2: 0.291865	valid_1's l2: 0.29543
[450]	training's l2: 0.290595	valid_1's l2: 0.295229
[500]	training's l2: 0.289375	valid_1's l2: 0.295072
[550]	training's l2: 0.288242	valid_1's l2: 0.294965
[600]	training's l2: 0.287176	valid_1's l2: 0.294855
[650]	training's l2: 0.286095	valid_1's l2: 0.294728
[700]	training's l2: 0.285137	valid_1's l2: 0.294699
[750]	training's l2: 0.284153	valid_1's l2: 0.294669
[800]	training's l2: 0.283234	valid_1's l2: 0.294656
[850]	training's l2: 0.282314	valid_1's l2: 0.29464
[900]	train

Run Time: 0:02:32.446015
Step 3
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.419072	valid_1's l2: 0.41755
[100]	training's l2: 0.335904	valid_1's l2: 0.340633
[150]	training's l2: 0.321125	valid_1's l2: 0.328992
[200]	training's l2: 0.316447	valid_1's l2: 0.32607
[250]	training's l2: 0.313733	valid_1's l2: 0.324841
[300]	training's l2: 0.3116	valid_1's l2: 0.324073
[350]	training's l2: 0.309886	valid_1's l2: 0.32362
[400]	training's l2: 0.308443	valid_1's l2: 0.3234
[450]	training's l2: 0.307141	valid_1's l2: 0.323211
[500]	training's l2: 0.305926	valid_1's l2: 0.323036
[550]	training's l2: 0.304738	valid_1's l2: 0.322928
[600]	training's l2: 0.303613	valid_1's l2: 0.322871
[650]	training's l2: 0.302483	valid_1's l2: 0.322763
[700]	training's l2: 0.301446	valid_1's l2: 0.322729
[750]	training's l2: 0.300404	valid_1's l2: 0.322674
[800]	training's l2: 0.299383	valid_1's l2: 0.322639
[850]	training's l2: 0.298422	valid_1's l2: 0.322647
[900]	trainin

...

Run Time: 0:03:07.361332
Step 12
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.503986	valid_1's l2: 0.485358
[100]	training's l2: 0.394058	valid_1's l2: 0.390557
[150]	training's l2: 0.373617	valid_1's l2: 0.377187
[200]	training's l2: 0.366748	valid_1's l2: 0.374199
[250]	training's l2: 0.362504	valid_1's l2: 0.373012
[300]	training's l2: 0.359005	valid_1's l2: 0.371532
[350]	training's l2: 0.356259	valid_1's l2: 0.370726
[400]	training's l2: 0.354124	valid_1's l2: 0.370162
[450]	training's l2: 0.352303	valid_1's l2: 0.369836
[500]	training's l2: 0.350695	valid_1's l2: 0.369606
[550]	training's l2: 0.349269	valid_1's l2: 0.369512
[600]	training's l2: 0.347851	valid_1's l2: 0.369365
[650]	training's l2: 0.346532	valid_1's l2: 0.369286
[700]	training's l2: 0.345233	valid_1's l2: 0.36919
[750]	training's l2: 0.344025	valid_1's l2: 0.369156
[800]	training's l2: 0.342819	valid_1's l2: 0.369098
[850]	training's l2: 0.34161	valid_1's l2: 0.369042
[900]	t

Run Time: 0:02:23.822133
Step 13
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.527013	valid_1's l2: 0.505117
[100]	training's l2: 0.407054	valid_1's l2: 0.400757
[150]	training's l2: 0.38418	valid_1's l2: 0.38552
[200]	training's l2: 0.37622	valid_1's l2: 0.382356
[250]	training's l2: 0.371476	valid_1's l2: 0.381067
[300]	training's l2: 0.367684	valid_1's l2: 0.379815
[350]	training's l2: 0.364634	valid_1's l2: 0.37897
[400]	training's l2: 0.362394	valid_1's l2: 0.378632
[450]	training's l2: 0.360304	valid_1's l2: 0.378273
[500]	training's l2: 0.35842	valid_1's l2: 0.378039
[550]	training's l2: 0.356678	valid_1's l2: 0.377855
[600]	training's l2: 0.355122	valid_1's l2: 0.377874
[650]	training's l2: 0.353644	valid_1's l2: 0.377742
[700]	training's l2: 0.352246	valid_1's l2: 0.377749
[750]	training's l2: 0.350914	valid_1's l2: 0.377683
[800]	training's l2: 0.349637	valid_1's l2: 0.377638
[850]	training's l2: 0.348391	valid_1's l2: 0.377574
[900]	trai

Run Time: 0:02:22.940212
Step 14
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.488912	valid_1's l2: 0.490503
[100]	training's l2: 0.393772	valid_1's l2: 0.40144
[150]	training's l2: 0.375743	valid_1's l2: 0.387409
[200]	training's l2: 0.369658	valid_1's l2: 0.384373
[250]	training's l2: 0.365638	valid_1's l2: 0.382958
[300]	training's l2: 0.362589	valid_1's l2: 0.381958
[350]	training's l2: 0.360227	valid_1's l2: 0.381301
[400]	training's l2: 0.358279	valid_1's l2: 0.380949
[450]	training's l2: 0.356481	valid_1's l2: 0.380661
[500]	training's l2: 0.354881	valid_1's l2: 0.380553
[550]	training's l2: 0.353383	valid_1's l2: 0.380366
[600]	training's l2: 0.351992	valid_1's l2: 0.380303
[650]	training's l2: 0.350632	valid_1's l2: 0.380266
[700]	training's l2: 0.349312	valid_1's l2: 0.380148
[750]	training's l2: 0.34806	valid_1's l2: 0.380033
[800]	training's l2: 0.346879	valid_1's l2: 0.380004
[850]	training's l2: 0.345687	valid_1's l2: 0.379981
[900]	t

Run Time: 0:02:05.363536
Step 15
Training until validation scores don't improve for 125 rounds
[50]	training's l2: 0.470656	valid_1's l2: 0.465708
[100]	training's l2: 0.381054	valid_1's l2: 0.383166
[150]	training's l2: 0.363963	valid_1's l2: 0.370282
[200]	training's l2: 0.358095	valid_1's l2: 0.367423
[250]	training's l2: 0.354314	valid_1's l2: 0.366318
[300]	training's l2: 0.351312	valid_1's l2: 0.365353
[350]	training's l2: 0.349103	valid_1's l2: 0.364782
[400]	training's l2: 0.347255	valid_1's l2: 0.364425
[450]	training's l2: 0.345579	valid_1's l2: 0.364163
[500]	training's l2: 0.344059	valid_1's l2: 0.364055
[550]	training's l2: 0.342658	valid_1's l2: 0.363928
[600]	training's l2: 0.341339	valid_1's l2: 0.363822
[650]	training's l2: 0.340104	valid_1's l2: 0.363721
[700]	training's l2: 0.338869	valid_1's l2: 0.363629
[750]	training's l2: 0.337696	valid_1's l2: 0.363632
[800]	training's l2: 0.3365	valid_1's l2: 0.363618
[850]	training's l2: 0.335347	valid_1's l2: 0.363595
[900]	t

Run Time: 0:02:53.774235


In [40]:
print("Validation mse:", mean_squared_error(
    y_val, np.array(val_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
val_err = (y_val - np.array(val_pred).transpose())**2
val_err = val_err.sum(axis=1) * weight
val_err = np.sqrt(val_err.sum() / weight.sum() / 15)
print('nwrmsle = {}'.format(val_err))

Validation mse: 0.34224540976658513
nwrmsle = 0.5847986873442191


In [42]:
print("Test mse:", mean_squared_error(
    y_test, np.array(test_pred).transpose()))

weight = items["perishable"] * 0.25 + 1
test_err = (y_test - np.array(test_pred).transpose())**2
test_err = test_err.sum(axis=1) * weight
test_err = np.sqrt(test_err.sum() / weight.sum() / 15)
print('nwrmsle = {}'.format(test_err))

Test mse: 0.3570734314951243
nwrmsle = 0.5971471217194578


In [ ]:
# y_val = np.array(val_pred).transpose()
# df_val = pd.DataFrame(
#     y_val, index=df_2017.index,
#     columns=pd.date_range("2017-07-26", periods=16)
# ).stack().to_frame("unit_sales")
# df_val.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)
# df_val["unit_sales"] = np.clip(np.expm1(df_preds["unit_sales"]), 0, 1000)
# df_val.reset_index().to_csv('lgb_cv.csv', index=False)

In [ ]:
# #print("Making submission...")
# y_test = np.array(test_pred).transpose()
# df_preds = pd.DataFrame(
#     y_test, index=df_2017.index,
#     columns=pd.date_range("2017-08-16", periods=16)
# ).stack().to_frame("unit_sales")
# df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [ ]:
y_test = np.array(test_pred).transpose()
y_test.shape

In [ ]:
df_preds_test = pd.DataFrame(
    y_test, index=df_2017.index,
    columns=pd.date_range("2017-08-01", periods=15)
).stack().to_frame("unit_sales")

In [ ]:
df_preds_test.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [ ]:
df_preds_test.head()

In [ ]:
df_preds_test.loc[(1,96995),:]

In [ ]:
df_2017.head()

In [ ]:
df_preds_test = pd.read_csv('lgb_new_fe_project_date_2019-11-06.csv')

In [ ]:
df_preds_test.set_index(['store_nbr','item_nbr','date'],inplace=True)

In [ ]:
df_preds_test.head()

In [ ]:
df_true = df_2017[pd.date_range('2017-08-01','2017-08-15')].stack().to_frame('unit_sales')

In [ ]:
df_true_agg = df_true.groupby(['store_nbr','item_nbr'])['unit_sales'].sum().sort_values(ascending=False)

In [ ]:
df_true_agg[:20]

In [ ]:
store_item_list = df_true_agg.index[0:10]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
date_range = pd.date_range('2017-08-01',periods=15)

plt.figure(figsize=(15,20))
for i, store_item in enumerate(store_item_list, 1):
    
    store_nbr = store_item[0]
    item_nbr = store_item[1]
    d = {'true':list(np.expm1(df_true.loc[(store_nbr,item_nbr)].values)),
    'predicted':list(np.expm1(df_preds_test.loc[(store_nbr,item_nbr)].values))}

    temp_df = pd.DataFrame(d,index=date_range)
    
    plt.subplot(len(store_item_list)/2,2,i)
    plt.plot(temp_df.true)
    plt.plot(temp_df.predicted)
    plt.legend()
    plt.xticks(rotation=90)
    plt.ylim(0,)
    plt.axvline('2017-08-05',ls=':',c='r')
    plt.axvline('2017-08-06',ls=':',c='r')
    plt.axvline('2017-08-12',ls=':',c='r')
    plt.axvline('2017-08-13',ls=':',c='r')
    plt.title(f'Store Number: {store_nbr} Item Number: {item_nbr}')
plt.tight_layout()
plt.show()
    
    

In [ ]:
store_nbr = 45
item_nbr =2042941
df_2017_plot = df_2017.loc[(store_nbr,item_nbr),'2017-07-01':'2017-08-15'].to_frame('unit_sales')
plt.plot(np.expm1(df_2017_plot))
plt.xticks(rotation=90)
plt.axvline('2017-08-01',ls='--',c='r')

In [ ]:
holidays = pd.read_csv('../input/holidays_events.csv',parse_dates=True)

In [ ]:
holidays['date'] = pd.to_datetime(holidays['date'])

In [ ]:
holidays.set_index('date',inplace=True)

In [ ]:
holidays.loc['2017-07-01':'2017-08-20']

In [ ]:
df_preds_test.to_csv('lgb_new_fe_project_date_2019-11-06.csv', float_format='%.4f')

In [ ]:
# submission = df_test[["id"]].join(df_preds, how="left").fillna(0)
# submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
# submission.to_csv('lgb_new_fe_project_date_2019-11-06.csv', float_format='%.4f', index=None)